#### Nombre: **06_modelo_genero**
#### Autores: _Sergio Cañón  Laiz y Ignacio Ruiz de Zuazu Echavarría_
#### CUNEF
#### 15/01/2021


# Librerías

In [5]:
# importamos librerías 

import numpy as np  # álgebra lineal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import glob  # pathnames
import cv2  # lectura y procesamiento de las imagenes

import matplotlib.pyplot as plt  # gráficos
import matplotlib.image as mpimg

import os  # lectura de directorios

import keras 
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras import optimizers
import time #tiempos de ejecución
from random import shuffle
from keras.preprocessing import image
import imageio


from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.applications.resnet50 import preprocess_input

# Importación de datos

In [2]:
categorical_labels_gender = np.load(r'..\data\04_data_array\categorical_labels_gender.npy')

X = np.load(r'..\data\04_data_array\data_32_32_rgb.npy')

size = len(X)
print("La longuitud del los datos son de:", size)



La longuitud del los datos son de: 24099


Usaremos un 30% de las iamgenes como test y el 70% como train.


In [3]:
#creacion train y test set

(x_train, y_train), (x_test, y_test) = (X[:15008],categorical_labels_gender[:15008]) , (X[15008:] , categorical_labels_gender[15008:])
(x_valid , y_valid) = (x_test[:7000], y_test[:7000])
(x_test, y_test) = (x_test[7000:], y_test[7000:])

In [4]:
print("Número de imagenes en el train:",x_train.shape[0])
print("Número de imagenes en el test:",x_test.shape[0])

Número de imagenes en el train: 15008
Número de imagenes en el test: 2091


# Model Building

## SGD model

In [6]:
model_sgd_gender = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)

model_sgd_gender.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_sgd_gender.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_sgd_gender.add(tf.keras.layers.Dropout(0.3))

model_sgd_gender.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) 
model_sgd_gender.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# drop out= eliminamos el 30% de las imagenes para evitar overfitting
model_sgd_gender.add(tf.keras.layers.Dropout(0.3))

model_sgd_gender.add(tf.keras.layers.Flatten())
model_sgd_gender.add(tf.keras.layers.Dense(256, activation='relu'))

# drop out= eliminamos la mitad de las imagenes para evitar overfitting
model_sgd_gender.add(tf.keras.layers.Dropout(0.5)) 


# nuestro data set tiene 5 opciones de raza luego la última capa = 5

model_sgd_gender.add(tf.keras.layers.Dense(2, activation='softmax')) 


# Resumen del modelo
model_sgd_gender.summary()

model_sgd_gender.compile(loss='binary_crossentropy',
             optimizer=SGD(),
             metrics=['accuracy','AUC','categorical_accuracy'])

h_model_sgd_gender = model_sgd_gender.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       65664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0

In [7]:
os.chdir('C:\\Users\\Apuntes\\Desktop\\labUTKfaces\\models\\')

In [10]:
import json
model_sgd_gender_history = h_model_sgd_gender.history
json.dump(model_sgd_gender_history, open("../models/model_sgd_gender_history.json", 'w'))

In [13]:
np.save('x_test_genero.npy', x_test)
np.save('y_test_genero.npy', y_test)


In [12]:
model_sgd_gender.save('../models/model_sgd_gender_rgb.h')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/model_sgd_gender_rgb.h\assets


# Adam Model

In [14]:
model_adam_gender = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)
model_adam_gender.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_adam_gender.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_gender.add(tf.keras.layers.Dropout(0.3))

model_adam_gender.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) 
model_adam_gender.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_gender.add(tf.keras.layers.Dropout(0.3))

model_adam_gender.add(tf.keras.layers.Flatten())
model_adam_gender.add(tf.keras.layers.Dense(256, activation='relu'))
model_adam_gender.add(tf.keras.layers.Dropout(0.5))

model_adam_gender.add(tf.keras.layers.Dense(2, activation='softmax'))

# Take a look at the model summary
model_adam_gender.summary()

model_adam_gender.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy','AUC','categorical_accuracy'])

h_model_adam_gender = model_adam_gender.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       65664     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)             

In [15]:
import json
model_adam_gender_history = h_model_adam_gender.history
json.dump(model_adam_gender_history, open("../models/model_adam_gender_history.json", 'w'))

In [16]:
model_adam_gender.save('../models/model_adam_gender_rgb.h')

INFO:tensorflow:Assets written to: ../models/model_adam_gender_rgb.h\assets
